In [3]:
import itertools # 是 Python 的內建模組，提供了一組用於處理迭代器的函數和工具。
                 # 它包含了各種用於高效處理迭代器的函數，可以幫助我們編寫更簡潔、高效的代碼。
import sys # 是 Python 的內建模組，提供了與 Python 解釋器和運行環境相關的功能。
# sys.path 是一個列表，包含了 Python 解釋器在導入模組時會搜尋的路徑。
# 當你使用 import 語句導入模組時 Python 會依次在 sys.path 中的路徑下尋找對應的模組文件。
sys.path.append("../ecg-classification/")
# sys.path.append("C:\\Users\\Chen_Lab01\\Documents\\GitHub/ecg-classification")
# from IPython.display import Video
# import cv2
from matplotlib import pyplot as plt
plt.style.use("ggplot") #  是 Matplotlib 庫中用於設置繪圖樣式的函數。它使用了一種名為 "ggplot" 的預定義樣式
import matplotlib
matplotlib.use("TkAgg")
                        #  該樣式模仿了 R 語言的 ggplot2 繪圖包的外觀。
# print(sys.path)
from ecg_tools.config import EcgConfig, Mode
from ecg_tools.data_loader import DatasetConfig, get_data_loaders
from ecg_tools.model import ECGformer
from ecg_tools.train import ECGClassifierTrainer


Load model

In [12]:
import torch
config = EcgConfig()    
model_quantized = torch.load("..\\model_save\\model_quantized.pth")
model = torch.load("..\\model_save\\model.pth")

In [18]:
import einops
loader = get_data_loaders(DatasetConfig())
accuracy = 0
for signal, label in loader[Mode.eval]:
    p = model(einops.rearrange(signal, "b c e -> b e c"))
    print(p)
    print(label)
    # print(signal.shape, label.shape)
    print(p.argmax(1) == label)
    accuracy += torch.sum(p.argmax(1) == label)
    print(f"accuracy: {accuracy / config.dataset.batch_size}")
    break



tensor([[ 0.3242,  0.0634,  0.0545, -0.4082, -0.8732, -0.3882],
        [ 0.3400,  0.0696,  0.1121, -0.4515, -0.9633, -0.3807],
        [ 0.3253,  0.0738,  0.0581, -0.4306, -0.8820, -0.3873],
        ...,
        [ 0.3264,  0.0637,  0.0591, -0.4206, -0.8733, -0.3814],
        [ 0.3451,  0.0798,  0.0769, -0.4436, -0.8761, -0.3861],
        [ 0.3166,  0.0638,  0.0676, -0.4153, -0.8966, -0.3883]],
       grad_fn=<AddmmBackward0>)
tensor([2, 0, 0, 0, 0, 0, 0, 5, 0, 5, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 2, 0, 0, 4, 5, 0, 0, 0, 5, 0, 0, 0, 0, 0, 4, 0, 0, 2, 0, 0, 0, 1,
        0, 0, 0, 1, 0, 0, 0, 5, 0, 0, 5, 0, 0, 0, 0, 0, 1, 0, 4, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 4, 0, 5, 5, 0, 4, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0, 4, 5, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 4, 0, 2, 5, 2, 0, 0, 0, 2, 0, 5, 0, 0, 0, 0, 4, 0, 0, 0, 5, 0, 0,
        5, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
        0

## Parameter 提取

In [5]:
for name in model_quantized.state_dict():
   print(name)

print(model_quantized.state_dict()['encoder.0.0.block.1.queries_projection._packed_params._packed_params'])

positional_encoding
encoder.0.0.block.0.weight
encoder.0.0.block.0.bias
encoder.0.0.block.1.queries_projection.scale
encoder.0.0.block.1.queries_projection.zero_point
encoder.0.0.block.1.queries_projection._packed_params.dtype
encoder.0.0.block.1.queries_projection._packed_params._packed_params
encoder.0.0.block.1.values_projection.scale
encoder.0.0.block.1.values_projection.zero_point
encoder.0.0.block.1.values_projection._packed_params.dtype
encoder.0.0.block.1.values_projection._packed_params._packed_params
encoder.0.0.block.1.keys_projection.scale
encoder.0.0.block.1.keys_projection.zero_point
encoder.0.0.block.1.keys_projection._packed_params.dtype
encoder.0.0.block.1.keys_projection._packed_params._packed_params
encoder.0.0.block.1.final_projection.scale
encoder.0.0.block.1.final_projection.zero_point
encoder.0.0.block.1.final_projection._packed_params.dtype
encoder.0.0.block.1.final_projection._packed_params._packed_params
encoder.0.1.block.0.weight
encoder.0.1.block.0.bias
enco